In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from random import choice
from enbios2.base.experiment import Experiment
import bw2data

from enbios2.bw2.util import report
from enbios2.models.experiment_models import ExperimentData

In [3]:
report()

Project: default
['biosphere3']
Project: ecoinvent
['biosphere3', 'cutoff_3.9.1_default']


In [4]:
bw2data.projects.set_current("ecoinvent")
database_name = 'cutoff_3.9.1_default'
db = bw2data.Database(database_name)

In [5]:
wind_turbines_spain = db.search("electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"})
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, ES, None),
 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, ES, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, ES, None)]

In [6]:
experiment_activities = []
for activity in wind_turbines_spain:
    experiment_activities.append(
        {"id":
            {
                "name": activity["name"],
                "location": activity["location"],
                "code": activity["code"]
            }
        }
    )

In [7]:
experiment_activities[0]["output"] = ["kilowatt_hour", 3]
experiment_activities

[{'id': {'name': 'electricity production, wind, >3MW turbine, onshore',
   'location': 'ES',
   'code': '0d48975a3766c13e68cedeb6c24f6f74'},
  'output': ['kilowatt_hour', 3]},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'location': 'ES',
   'code': 'ed3da88fc23311ee183e9ffd376de89b'}},
 {'id': {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'location': 'ES',
   'code': '6ebfe52dc3ef5b4d35bb603b03559023'}},
 {'id': {'name': 'electricity production, wind, <1MW turbine, onshore',
   'location': 'ES',
   'code': '72cc067e1f4093c2e4c6ac9bdc93d844'}}]

In [8]:
all_methods = list(bw2data.methods)
methods = [choice(all_methods) for _ in range(2)]
methods

[('Ecological Scarcity 2021', 'natural resources', 'biotic resources'),
 ('EDIP 2003', 'global warming', 'GWP 100a')]

In [9]:
experiment_methods = [
    {
        "id": method
    }
    for method in methods
]

In [10]:
exp_data = ExperimentData(
    bw_project="ecoinvent",
    activities=experiment_activities,
    methods=experiment_methods
)

In [11]:
exp: Experiment = Experiment(exp_data)

In [16]:
exp.scenarios[0].result_tree[0].data

ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={}, bw_activity='electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None))

In [13]:
print(exp.info())

Experiment: 
Activities: 4
  electricity production, wind, >3MW turbine, onshore - electricity production, wind, >3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, onshore - electricity production, wind, 1-3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, offshore - electricity production, wind, 1-3MW turbine, offshore
  electricity production, wind, <1MW turbine, onshore - electricity production, wind, <1MW turbine, onshore
Methods: 2
 ('EF v3.0', 'climate change: fossil', 'global warming potential (GWP100)')
 ('EF v3.1', 'climate change: fossil', 'global warming potential (GWP100)')
Scenarios: 1


In [18]:
print(exp.scenarios[0].result_tree.info())

[root - 4 children ]
ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={}, bw_activity=None)


In [19]:
# run all scenarios at once
results = exp.run()

2023-08-02 15:12:57,517 - bw2calc - INFO - Initialized LCA object. Demand: {25415: 1, 15150: 1, 25751: 1, 25926: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x7f81c4e7c890>, <bw_processing.datapackage.Datapackage object at 0x7f81c46c7590>, <bw_processing.datapackage.Datapackage object at 0x7f81c470fc50>, <bw_processing.datapackage.Datapackage object at 0x7f81c46e1b90>, <bw_processing.datapackage.Datapackage object at 0x7f81c46e3990>, <bw_processing.datapackage.Datapackage object at 0x7f81c500ea10>]
2023-08-02 15:12:57,517 - bw2calc - INFO - {'message': 'Started MultiLCA calculation', 'methods': [('Ecological Scarcity 2021', 'natural resources', 'biotic resources'), ('EDIP 2003', 'global warming', 'GWP 100a')], 'functional units': [[{'database': 'cutoff_3.9.1_default', 'code': '0d48975a3766c13e68cedeb6c24f6f74', 'amount': 3.0}], [{'database': 'cutoff_3.9.1_default', 'code': 'ed3da88fc23311ee183e9ffd376de89b', 'amount': 1.0}], [{'database': 'cutoff_3.9.1_default', 'c

# Result
The result is a dictionary of scenario names, where for each scenario we have a tree (representing the activity hierarchy). Each node (`BasicTreeNode`) in the tree has a data object, which is of the type `ScenarioResultNodeData`, which have the fields `output`, `result` and `bw_activity`.

In [20]:
# from enbios2.generic.tree.basic_tree import BasicTreeNode
# from enbios2.models.experiment_models import ScenarioResultNodeData

results

{'default scenario': [root - 4 children ]}

In [21]:
print(results["default scenario"].info())
print("---")
for children in results["default scenario"]:
    print(children.info())
    print("---")

[root - 4 children ]
ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={'Ecological Scarcity 2021_natural resources_biotic resources': 3.250685705874655e-15, 'EDIP 2003_global warming_GWP 100a': 0.11572048496828669}, bw_activity=None)
---
[electricity production, wind, >3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={'Ecological Scarcity 2021_natural resources_biotic resources': 3.0990748530770947e-15, 'EDIP 2003_global warming_GWP 100a': 0.07132284990943508}, bw_activity='electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None))
---
[electricity production, wind, 1-3MW turbine, onshore - 0 children  (root)]
ScenarioResultNodeData(output=('kilowatt_hour', 1.0), results={'Ecological Scarcity 2021_natural resources_biotic resources': 1.4039679461985164e-16, 'EDIP 2003_global warming_GWP 100a': 0.014479886661438488}, bw_activity='electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour,

In [ ]:
from enbios2.models.experiment_models import ExperimentScenarioData

exp_data.scenarios = []

for _ in range(3):
    random_activity = choice(list(exp.activitiesMap.keys()))
    # print(random_activity)
    exp_data.scenarios.append(
        ExperimentScenarioData(activities={
            random_activity: [exp.activitiesMap[random_activity].output.unit, 3]
        })
    )

exp_data.scenarios